# Análise comparativa de modelos

In [ ]:
from IPython.display import Image, display, Markdown
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import joblib

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import ShuffleSplit, GridSearchCV, KFold, cross_validate
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

## 1. Obtenção de Dados

Nessa etapa obtemos novamente os arquivos brutos de dados e o dicionário antes de iniciar o pré-processamento.

In [ ]:
df = pd.read_csv("../data/raw/diabetes_prediction_dataset.csv")
df_dict = pd.read_csv("../data/external/dictionary.csv")
df_dict

## 2. Preparação de dados

Aqui realizamos a normalização, codificação e o tratamento de dados discrepantes e/ou faltantes dentro do conjunto de dados.

In [3]:
# Definindo a variável alvo e as colunas nominais, contínuas e discretas
target_column = 'diabetes'  # A variável alvo

# Filtrando as colunas nominais e contínuas
nominal_columns = (
    df_dict
    .query("subtipo == 'nominal' and variavel != @target_column")
    .variavel
    .to_list()
)

continuous_columns = (
    df_dict
    .query("subtipo == 'continua'")
    .variavel
    .to_list()
)

# Filtrando as colunas discretas
discrete_columns = (
    df_dict
    .query("subtipo == 'discreta'and variavel != @target_column")
    .variavel
    .to_list()
)

# Separando as features (X) da variável alvo (y)
X = df.drop(columns=[target_column], axis=1)
y = df[target_column]

## 2.1 Pré-processamento

In [ ]:
# Pré-processador para variáveis nominais
nominal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')),  # Tratamento de dados faltantes
    ('encoding', OneHotEncoder(sparse_output=False, drop='first')),  # Codificação de variáveis nominais
])

# Pré-processador para variáveis contínuas
continuous_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='mean')),  # Tratamento de dados faltantes
    ('normalization', StandardScaler())  # Normalização de dados contínuos
])

# Pré-processador para variáveis discretas
discrete_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')),  # Tratamento de dados faltantes
    ('normalization', MinMaxScaler())  # Normalização de variáveis discretas
])

# Combinando os pré-processadores usando ColumnTransformer
preprocessor = ColumnTransformer([
    ('nominal', nominal_preprocessor, nominal_columns),
    ('continuous', continuous_preprocessor, continuous_columns),
    ('discrete', discrete_preprocessor, discrete_columns)
])

# Aplicando o pré-processamento ao conjunto de dados
X_preprocessed = preprocessor.fit_transform(X)

# Verificando os resultados do pré-processamento
print("X_preprocessed shape:", X_preprocessed.shape)
print("X_preprocessed sample:", X_preprocessed[:5])

## 3. Seleção de modelos

### 3.1. Metodologia

Iremos análisar quatro modelos, que serão testados utilizando um método de validação, a saber:

- K-Nearest-Neighbors
- Support Vector Machine
- Decision Tree
- Random Forest
Além disso, cada um desses algoritmos será testado com diferentes hiper-parametros, para que possamos encontrar o melhor modelo e a melhor configuração possível para esse modelo.

Utilizaremos as seguintes métricas para análise:

- **Acurácia (accuracy)**: proporção entre os dados que foram corretamente previstos (como positivos ou negativos) com o total de dados observados;
- **Precisão (precision)**: proporção entre dados corretamente previstos como positivos e o total de observações positivas.
- **Recall**: proporção entre dados corretamente previstos como positivos com o total de observações.
- **F1-score**: média entre precision e recall, portanto levando em conta tanto falsos positivos quanto falsos negativos.

### 3.2. Configuração do experimento

In [5]:
# Configurações do experimento
n_splits_comparative_analysis = 10
n_folds_grid_search = 5
test_size = 0.3
random_state = 0
scoring = 'accuracy'

# Métricas para análises
metrics = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

# Configurações do modelo
max_iter = 10000000
models = [
    ('K-Nearest Neighbors', KNeighborsClassifier(), {"n_neighbors": range(3, 20, 2), 'weights': ['uniform', 'distance']}),
    ('Decision Tree',  DecisionTreeClassifier(random_state=random_state), {'criterion':['gini','entropy'],'max_depth': [3, 6, 8]}),
    ('Random Forest',  RandomForestClassifier(random_state=random_state), {'criterion':['gini','entropy'],'max_depth': [3, 6, 8], 'n_estimators': [10, 30]}),
    ('Suport Vector Machines', SVC(random_state=random_state, max_iter=max_iter), {"kernel": ["linear", "rbf"], 'C':[1,10,100,1000],'gamma':[0.0001, 0.001, 0.1, 1]})
]

In [ ]:
results = pd.DataFrame({})
cross_validate_grid_search = KFold(n_splits=n_folds_grid_search)
cross_validate_comparative_analysis = ShuffleSplit(n_splits=n_splits_comparative_analysis, test_size=test_size, random_state=random_state)
for model_name, model_object, model_parameters in models:
    print(f"running {model_name}...")
    model_grid_search = GridSearchCV(
        estimator=model_object,
        param_grid=model_parameters,
        scoring=scoring,
        n_jobs=-1,
        cv=cross_validate_grid_search
    )
    approach = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model_grid_search)
    ])
    scores = cross_validate(
        estimator=approach,
        X=X,
        y=y,
        cv=cross_validate_comparative_analysis,
        n_jobs=-1,
        scoring=metrics
    )
    scores['model_name'] = [model_name] * n_splits_comparative_analysis
    df_scores = pd.DataFrame(scores)
    df_scores =  df_scores.drop(columns=['model_name'])
    df_scores = df_scores.agg(['mean', 'std'])
    display(df_scores)
    results = pd.concat([results, pd.DataFrame(scores)], ignore_index=True)

running K-Nearest Neighbors...
